In [0]:
%%sql
SELECT
    count(*) as num_records,
    date_trunc(:time_unit, timestamp) as time_bucket,

    CASE
        WHEN :time_unit = 'second' THEN :target_rate::double
        WHEN :time_unit = 'minute' THEN :target_rate::double * 60
        WHEN :time_unit = 'hour' THEN :target_rate::double * 3600
        WHEN :time_unit = 'day' THEN :target_rate::double * 86400
        ELSE :target_rate::double
    END as target_rate

FROM read_kafka(
        `kafka.bootstrap.servers` => secret(:secret_scope, 'KAFKA_BROKERS'),
        `kafka.security.protocol` => 'SASL_SSL',
        `kafka.sasl.mechanism` => 'SCRAM-SHA-512',
        `kafka.sasl.jaas.config` => CONCAT(
        'kafkashaded.org.apache.kafka.common.security.scram.ScramLoginModule required username="',
        secret(:secret_scope, 'KAFKA_USERNAME'),
        '" password="',
        secret(:secret_scope, 'KAFKA_PASSWORD'),
        '";'
        ),
        `subscribe` => :topic_name
)

where 
    timestamp >= :starting_from_time

group by all